In [2]:
import torch

from pathlib import Path

from app.model.config import Config
from app.model.time_series_embedding import TimeSeriesEmbedding

data_dir = Path('app/data')
config = Config(data_dir)
time_series_embedding = TimeSeriesEmbedding(config)
input = torch.randn(32, config.seq_len, config.input_dims)
output = time_series_embedding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 5]); output.shape=torch.Size([32, 5, 150])'

In [3]:
import torch
from app.model.config import Config
from app.model.positional_encoding import PositionalEncoding

data_dir = Path('app/data')
config = Config(data_dir)
positional_encoding = PositionalEncoding(config)
input = torch.randn((32, config.seq_len, config.d_model))
output = positional_encoding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 150]); output.shape=torch.Size([32, 5, 150])'

In [4]:
import torch
from app.model.config import Config
from app.model.stock_transformer import StockTransformer


data_dir = Path('app/data')
config = Config(data_dir)
input = torch.randn((config.batch_size, config.seq_len, config.input_dims),
                    dtype=torch.float)
display(f'{input.shape=}')
transformer = StockTransformer(config)
output = transformer(input)
display(f'{output.shape=}')

'input.shape=torch.Size([16, 5, 5])'

'output.shape=torch.Size([16, 1])'

In [4]:
import csv
import yfinance as yf

from pathlib import Path


ticker = 'SPY'
data = yf.Ticker(ticker)
hist = data.history(period='max', interval='1d')

if hist.empty:
    print(f'No data available for {ticker}')
else:
    date_from = hist.index.min().strftime('%Y%m%d')
    date_to = hist.index.max().strftime('%Y%m%d')

    out_dir = Path('app/data') / 'tickers'
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{ticker}_{date_from}_{date_to}.csv"

    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    with out_path.open('w', newline='') as fp:
        writer = csv.writer(fp)
        writer.writerow(columns)
        for row in hist.itertuples():
            writer.writerow([getattr(row, c) for c in columns])

    print(f'Saved {out_path}')

Saved app/data/tickers/SPY_19930129_20251231.csv


In [70]:
import torch
import pandas as pd

from pathlib import Path

from app.model.checkpointer import Checkpointer
from app.model.stock_dataset import StockDataset


model_path = Path('app/data/models/4000/SPY.pkl')
model, config, step = Checkpointer.load_checkpoint(model_path)
dataset = StockDataset(config)
input, target = dataset.__getitem__(0)
output = model(input)
last_close = input[-1, -3]
predicted_log_return = output[0][0]
predicted_close = last_close * torch.exp(predicted_log_return)
display(last_close.item(), predicted_close.item())

Model loaded from app/data/models/4000/SPY.pkl


24.82761573791504

25.083288192749023